In [ ]:
from datascience import *
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline

## Flight delays; a single sample

In [ ]:
united = Table.read_table('http://inferentialthinking.com/data/united_summer2015.csv')

In [ ]:
la = united.where('Destination', 'LAX')

In [ ]:
sample_from_population = la.sample(100, with_replacement=False)

In [ ]:
np.median(sample_from_population['Delay'])

## Variability among samples

In [ ]:
medians = []
repetitions = np.arange(200)
for i in repetitions: # examine 200 samples
    sample = la.sample(100, with_replacement=False)
    medians.append(np.median(sample['Delay']))

Table().with_columns('i', repetitions, 'median', medians).scatter(0)

In [ ]:
Table().with_columns('median', medians).hist(0)

In [ ]:
percentile(2.5, medians)

In [ ]:
percentile(97.5, medians)

In [ ]:
print("95% of sample medians were between", 
      percentile(2.5, medians),
      "and",
      percentile(97.5, medians))

## Predicting the accuracy of our estimate -- the bootstrap

In [ ]:
sample_from_population # from now on, this is all we have

In [ ]:
np.median(sample_from_population['Delay'])

In [ ]:
resample_1 = sample_from_population.sample()
resample_1.show(3)

In [ ]:
np.median(resample_1['Delay'])

In [ ]:
resample_2 = sample_from_population.sample()
resample_2.show(3)

In [ ]:
np.median(resample_2['Delay'])

In [ ]:
resampled_medians = make_array()
for i in np.arange(5000):
    resample = sample_from_population.sample()
    median = np.median(resample['Delay'])
    resampled_medians = np.append(resampled_medians, median)
    
Table().with_column('Resampled median', resampled_medians).hist(0)

## Confidence intervals

In [ ]:
print("95% of resampled medians were between", 
      percentile(2.5, resampled_medians),
      "and",
      percentile(97.5, resampled_medians))

In [ ]:
interval_95 = [percentile(2.5, resampled_medians),
               percentile(97.5, resampled_medians)]

Table().with_column('Resampled median', resampled_medians).hist(0)
plots.plot(interval_95, [0, 0], color='gold', lw=5)
pop_median = np.median(la['Delay'])
plots.scatter(pop_median, 0, color='red', s=400)

In [ ]:
def bootstrap_median(sample_from_pop):
    resampled_medians = make_array()
    for i in np.arange(5000):
        resample = sample_from_pop.sample()
        median = np.median(resample['Delay'])
        resampled_medians = np.append(resampled_medians, median)
    return resampled_medians

def plot_bootstrap(sample_from_pop):
    resampled_medians = bootstrap_median(sample_from_pop)
    interval_95 = [percentile(2.5, resampled_medians),
                   percentile(97.5, resampled_medians)]
    Table().with_column('Resampled median', resampled_medians).hist(0)
    plots.plot(interval_95, [0, 0], color='gold', lw=5)
    pop_median = np.median(la['Delay'])
    plots.scatter(pop_median, 0, color='red', s=400)

In [ ]:
different_sample_from_population = la.sample(100)
print('Sample median:', np.median(different_sample_from_population['Delay']))

In [ ]:
plot_bootstrap(different_sample_from_population)

In [ ]:
# THE BIG SIMULATION: This one takes several minutes.

# Generate 100 intervals, in the table intervals

left_ends = make_array()
right_ends = make_array()

for i in np.arange(100):
    sample_from_pop = la.sample(100, with_replacement=False)
    medians = bootstrap_median(sample_from_pop)
    left_ends = np.append(left_ends, percentile(2.5, medians))
    right_ends = np.append(right_ends, percentile(97.5, medians))

intervals = Table().with_columns(
    'Left', left_ends,
    'Right', right_ends
)

In [ ]:
intervals

In [ ]:
intervals.where('Left', are.below(pop_median)).where('Right', are.above(pop_median)).num_rows

In [ ]:
replication_number = np.ndarray.astype(np.arange(1, 101), str)

intervals2 = Table(replication_number).with_rows(make_array(left_ends, right_ends))

plots.figure(figsize=(8,8))
n=100
for i in np.arange(n):
    ends = intervals2.column(i)
    plots.plot(ends, make_array(i+1, i+1), color='gold')
plots.plot(make_array(pop_median, pop_median), make_array(0, n), color='red', lw=2)
plots.xlabel('Median (minutes)')
plots.ylabel('Replication')
plots.title('Population Median and Intervals of Estimates');